In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import matplotlib.pyplot as plt
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [3]:
# [2]번줄 코드 해석 :
# MNIST라는 숫자 데이터셋을 다운받기 위하여, torchvison 이라는 라이브러를 미리 다운로드 한다.
# 가지고 있는 데이터의 순서를 섞거나 원하는 비율로 나누거나 하는 데이터를 전처리르 위해 DataLoader를 선언한다.

In [ ]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10